<a href="https://colab.research.google.com/github/dcpatton/Image-Classification/blob/master/cifar10_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import torch
import torchvision
import torchvision.transforms as transforms

In [55]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [56]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [57]:
len(trainset.targets)

50000

In [58]:
len(testset.targets)

10000

In [59]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [60]:
net.to(device)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [61]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr = 0.001)

In [62]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # inputs, labels = data
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.834
[1,  4000] loss: 1.580
[1,  6000] loss: 1.504
[1,  8000] loss: 1.455
[1, 10000] loss: 1.390
[1, 12000] loss: 1.361
[2,  2000] loss: 1.288
[2,  4000] loss: 1.287
[2,  6000] loss: 1.242
[2,  8000] loss: 1.257
[2, 10000] loss: 1.239
[2, 12000] loss: 1.236
[3,  2000] loss: 1.152
[3,  4000] loss: 1.147
[3,  6000] loss: 1.141
[3,  8000] loss: 1.141
[3, 10000] loss: 1.132
[3, 12000] loss: 1.098
[4,  2000] loss: 1.049
[4,  4000] loss: 1.053
[4,  6000] loss: 1.043
[4,  8000] loss: 1.047
[4, 10000] loss: 1.061
[4, 12000] loss: 1.050
[5,  2000] loss: 0.985
[5,  4000] loss: 0.992
[5,  6000] loss: 1.007
[5,  8000] loss: 1.015
[5, 10000] loss: 0.980
[5, 12000] loss: 0.995
[6,  2000] loss: 0.946
[6,  4000] loss: 0.941
[6,  6000] loss: 0.947
[6,  8000] loss: 0.966
[6, 10000] loss: 0.955
[6, 12000] loss: 0.956
[7,  2000] loss: 0.902
[7,  4000] loss: 0.931
[7,  6000] loss: 0.903
[7,  8000] loss: 0.929
[7, 10000] loss: 0.957
[7, 12000] loss: 0.932
[8,  2000] loss: 0.853
[8,  4000] 

In [63]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [64]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [65]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 66 %
